In [5]:
import os
import pandas as pd
import xarray as xr
from pathlib import Path

In [6]:

# Define the base directory containing NC files
base_dir = Path("../tests/tests_data/hazards")

# Find all NC files recursively
nc_files = list(base_dir.rglob("*.nc"))

print(f"Found {len(nc_files)} NC files to convert")

# Convert each NC file to CSV
for nc_file in nc_files:
    try:
        print(f"Converting: {nc_file}")
        
        # Open the NetCDF file
        ds = xr.open_dataset(nc_file)
        
        # Convert to DataFrame and clean up
        df = ds.to_dataframe().dropna().reset_index()
        
        # Extract hazard indicator from folder path (e.g., "HI" from "hazards/Compound/HI/ensemble/...")
        path_parts = nc_file.parts
        hazard_indicator = None
        
        # Look for the indicator name in the path structure
        # Expected structure: .../hazards/HazardType/IndicatorName/ensemble/...
        if 'hazards' in path_parts:
            hazards_index = path_parts.index('hazards')
            if hazards_index + 2 < len(path_parts):
                hazard_indicator = path_parts[hazards_index + 2]  # Skip 'hazards' and hazard type
                print(f"  -> Extracted hazard indicator '{hazard_indicator}' from path")
        
        # Get the data variable name (should be the only data variable in the dataset)
        data_vars = [var for var in ds.data_vars]
        if len(data_vars) == 1:
            data_var_name = data_vars[0]
            
            # Use extracted hazard indicator or fall back to data variable name
            indicator_name = hazard_indicator if hazard_indicator else data_var_name
            
            # Create new columns: hazard_indicator and hazard_intensity
            df['hazard_indicator'] = indicator_name
            df['hazard_intensity'] = df[data_var_name]
            
            # Remove the original data variable column
            df = df.drop(columns=[data_var_name])
            
            print(f"  -> Data variable '{data_var_name}' restructured using indicator '{indicator_name}'")
        else:
            print(f"  -> Warning: Found {len(data_vars)} data variables, expected 1. Using original structure.")
        
        # Create output CSV file path (same directory, same name, .csv extension)
        csv_file = nc_file.with_suffix('.csv')
        
        # Save as CSV
        df.to_csv(csv_file, index=False)
        
        print(f"  -> Saved as: {csv_file}")
        
        # Close the dataset to free memory
        ds.close()
        
    except Exception as e:
        print(f"Error converting {nc_file}: {e}")

print("Conversion complete!")

Found 2 NC files to convert
Converting: ../tests/tests_data/hazards/Compound/HI/ensemble/ensemble_return_period.nc
  -> Extracted hazard indicator 'HI' from path
  -> Data variable 'HI_max' restructured using indicator 'HI'
  -> Saved as: ../tests/tests_data/hazards/Compound/HI/ensemble/ensemble_return_period.csv
Converting: ../tests/tests_data/hazards/Drought/SPI6/ensemble/ensemble_return_period.nc
  -> Extracted hazard indicator 'SPI6' from path
  -> Data variable 'SPI6' restructured using indicator 'SPI6'
  -> Saved as: ../tests/tests_data/hazards/Drought/SPI6/ensemble/ensemble_return_period.csv
Conversion complete!
